<a href="https://colab.research.google.com/github/wenqiglantz/nvidia-sec-finetuning/blob/main/nvidia_sec_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install llama_index==0.8.16 pypdf sentence-transformers ragas

In [78]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    Response
)
from llama_index.evaluation import (
    DatasetGenerator,
    QueryResponseEvaluator,
    ResponseEvaluator
)
from llama_index.llms import OpenAI
import pandas as pd
import openai
import os

In [79]:
os.environ["OPENAI_API_KEY"] = "sk-############"
openai.api_key = os.environ["OPENAI_API_KEY"]

#define LLM
llm = OpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [80]:
!curl https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf --output nvidia-sec-10k-2022.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1541k  100 1541k    0     0  6899k      0 --:--:-- --:--:-- --:--:-- 6911k


In [81]:
# Shuffle the documents
import random

# load documents
documents = SimpleDirectoryReader(input_files=["nvidia-sec-10k-2022.pdf"]).load_data()
print(f"loaded documents with {len(documents)} pages")

random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

loaded documents with 169 pages


## Generate datasets

Let's first generate two datasets, one for training, the other for eval

### Training dataset

In [83]:
import random
random.seed(42)

questions = []
if os.path.exists("train_questions.txt"):
    with open("train_questions.txt", "r") as f:
        for line in f:
            questions.append(line.strip())
else:
    question_gen_query = (
        "You are a financial expert. You are asked to generate a list of questions "
        "using the provided context. Restrict the question to the context information provided."
    )
    # generate questions
    dataset_generator = DatasetGenerator.from_documents(
        documents[:50],
        question_gen_query=question_gen_query,
        service_context=gpt_35_context,
    )

    questions = dataset_generator.generate_questions_from_nodes(num=40)
    print(f"Generated {len(questions)} questions.")

    # save the questions!
    with open("train_questions.txt", "w") as f:
        for question in questions:
            f.write(f"{question.strip()}\n")

Generated 40 questions.


In [84]:
for i, question in enumerate(questions, start=1):
    print(f"{i}. {question}")

1. What factors could result in less demand for existing products?
2. How could new or unexpected end use cases impact demand for products?
3. What potential competitive actions could increase demand for competitive products?
4. How could business decisions made by third parties affect demand for products?
5. What factors could impact the demand for accelerated or AI-related cloud services?
6. How could the demand for cryptocurrency mining impact product demand?
7. What potential government actions or changes in policies could affect gaming usage and demand?
8. What factors have contributed to the significant growth in supply for the company?
9. How could misalignment between inventory/supply commitments and product demand result in inventory provisions?
10. How have product transitions negatively impacted the company's revenue in the past?
11. What challenges are associated with architecture transitions for Data Center, Professional Visualization, and Gaming products?
12. How could cu

### Eval dataset

In [85]:
questions = []
if os.path.exists("eval_questions.txt"):
    with open("eval_questions.txt", "r") as f:
        for line in f:
            questions.append(line.strip())
else:
    dataset_generator = DatasetGenerator.from_documents(
        documents[
            50:
        ],  # since we generated ~1 question for 40 documents, we can skip the first 40
        question_gen_query=question_gen_query,
        service_context=gpt_35_context,
    )

    questions = dataset_generator.generate_questions_from_nodes(num=40)
    print(f"Generated {len(questions)} questions.")

    # save the questions!
    with open("eval_questions.txt", "w") as f:
        for question in questions:
            f.write(f"{question.strip()}\n")

Generated 40 questions.


In [86]:
    for i, question in enumerate(questions, start=1):
        print(f"{i}. {question}")

1. How does the company recognize the benefit from a tax position?
2. What is the company's policy regarding interest and penalties related to unrecognized tax benefits?
3. How is basic net income per share computed?
4. How is diluted net income per share computed?
5. How does the company classify cash equivalents and marketable securities?
6. What is the classification of investments based on their nature and availability for use in current operations?
7. How are available-for-sale debt securities reported and valued?
8. How are realized gains and losses on the sale of marketable securities recorded?
9. What is the periodic impairment review process for available-for-sale debt investments?
10. How are allowances for credit losses and write-downs recognized?
11. How does the company determine the fair value of financial instruments?
12. What is the accounting treatment for derivative instruments designated as fair value hedges?
13. What is the accounting treatment for derivative instru

## Baseline eval for gpt-3.5-turbo

Let's evaluate our base model with both ragas framework and evaluation module


### Eval with ragas

In [87]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3), context_window=2048
)

# build vector index and query engine
index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)
query_engine = index.as_query_engine(similarity_top_k=2)


In [88]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [89]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)
result.to_pandas()

evaluating with [answer_relevancy]


100%|██████████| 3/3 [01:08<00:00, 22.70s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [03:47<00:00, 75.95s/it]


{'ragas_score': 0.8874, 'answer_relevancy': 0.9623, 'faithfulness': 0.8233}


,question,answer,contexts,answer_relevancy,faithfulness
0,How does the company recognize the benefit fro...,The company recognizes the benefit from a tax ...,[Table of Contents\nprice basis by maximizing ...,0.909524,1.000000
1,What is the company's policy regarding interes...,The company's policy is to include interest an...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,1.000000,1.000000
2,How is basic net income per share computed?,Basic net income per share is computed by divi...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.989098,1.000000
3,How is diluted net income per share computed?,Diluted net income per share is computed by di...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.993161,1.000000
4,How does the company classify cash equivalents...,The company classifies cash equivalents as hig...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.963275,0.666667
5,What is the classification of investments base...,The classification of investments based on the...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.854538,0.000000
6,How are available-for-sale debt securities rep...,Available-for-sale debt securities are reporte...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.940233,0.800000
7,How are realized gains and losses on the sale ...,Realized gains and losses on the sale of marke...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.986538,0.000000
8,What is the periodic impairment review process...,The periodic impairment review process for ava...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.943829,0.800000
9,How are allowances for credit losses and write...,Allowances for credit losses and write-downs a...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.926925,1.000000


### Eval with evaluation module


In [90]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()

def evaluate_query_engine(evaluator, query_engine, questions):
    async def run_query(query_engine, q):
        try:
            return await query_engine.aquery(q)
        except:
            return Response(response="Error, query failed.")

    total_correct = 0
    all_results = []
    for batch_size in range(0, len(questions), 5):
        batch_qs = questions[batch_size:batch_size+5]

        tasks = [run_query(query_engine, q) for q in batch_qs]
        responses = asyncio.run(asyncio.gather(*tasks))
        print(f"finished batch {(batch_size // 5) + 1} out of {len(questions) // 5}")

        # eval for hallucination
        if isinstance(evaluator, ResponseEvaluator):
          for response in responses:
              eval_result = 1 if "YES" in evaluator.evaluate(response) else 0
              total_correct += eval_result
              all_results.append(eval_result)
        # eval for answer quality
        elif isinstance(evaluator, QueryResponseEvaluator):
          for question, response in zip(batch_qs, responses):
              eval_result = 1 if "YES" in evaluator.evaluate(question, response) else 0
              total_correct += eval_result
              all_results.append(eval_result)

        # helps avoid rate limits
        time.sleep(1)

    return total_correct, all_results

In [92]:
# use gpt-4 to evaluate
gpt4_service_context = ServiceContext.from_defaults(llm=OpenAI(temperature=0.3, llm="gpt-4"))

questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [93]:
# eval for hallucination
evaluator = ResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Hallucination? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8


finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Hallucination? Scored 16 out of 40 questions correctly.


In [94]:
# eval for answer quality
evaluator = QueryResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Response satisfies the query? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Response satisfies the query? Scored 26 out of 40 questions correctly.


## GPT4 to collect training data

In [95]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [96]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [97]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)
query_engine = index.as_query_engine(similarity_top_k=2)

for question in questions:
    response = query_engine.query(question)

In [98]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 65 examples to finetuning_events.jsonl


## Create OpenAIFinetuneEngine

In [99]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events.jsonl"
)

In [100]:
finetune_engine.finetune()

Num examples: 65
First example:
{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}
{'role': 'user', 'content': "Context information is below.\n---------------------\npage_label: 19\nfile_name: nvidia-sec-10k-2022.pdf\n\nTable of Contents\n•new product introductions and transitions resulting in less demand for existing products;\n•new or unexpected end use cases;\n•increase in demand for competitive products, including competitive actions;\n•business decisions made by third parties;\n•the demand for accelerated or AI-related cloud services, including our own software and AI cloud service offerings;\n•the demand for cryptocurrency mining; or\n•government 

In [101]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-nHNw7m6f0HzuQ5B2Ia63uW8o at 0x78377a55be20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-nHNw7m6f0HzuQ5B2Ia63uW8o",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693717192,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-5ytHcLCFlcB1xR8qyZkYwLRd",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-duTvJZ60uJVniLgVrbrHmBb7",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [103]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

## Evaluation for fine-tuned model

### Eval with ragas

In [104]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

In [105]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [106]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [107]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [108]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)
result.to_pandas()

evaluating with [answer_relevancy]


100%|██████████| 3/3 [01:03<00:00, 21.23s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [05:15<00:00, 105.27s/it]


{'ragas_score': 0.8218, 'answer_relevancy': 0.9498, 'faithfulness': 0.7242}


,question,answer,contexts,answer_relevancy,faithfulness
0,How does the company recognize the benefit fro...,The company recognizes the benefit from a tax ...,[Table of Contents\nprice basis by maximizing ...,0.875141,0.666667
1,What is the company's policy regarding interes...,The company's policy is to include interest an...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.999999,1.000000
2,How is basic net income per share computed?,Basic net income per share is computed by divi...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.991540,1.000000
3,How is diluted net income per share computed?,Diluted net income per share is computed using...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.962701,0.666667
4,How does the company classify cash equivalents...,The company classifies cash equivalents as hig...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,1.000000,0.500000
5,What is the classification of investments base...,The context does not provide information on th...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.739432,1.000000
6,How are available-for-sale debt securities rep...,The valuation and reporting of available-for-s...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.919255,1.000000
7,How are realized gains and losses on the sale ...,Realized gains and losses on the sale of marke...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.949693,0.500000
8,What is the periodic impairment review process...,The context does not provide information on th...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,0.958285,1.000000
9,How are allowances for credit losses and write...,The context does not provide information on ho...,[Table of Contents\nNVIDIA CORPORATION AND SUB...,1.000000,0.000000


### Eval with evaluation module

In [109]:
# eval for hallucination
evaluator = ResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Hallucination? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Hallucination? Scored 16 out of 40 questions correctly.


In [110]:
# eval for answer quality
evaluator = QueryResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Response satisfies the query? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Response satisfies the query? Scored 22 out of 40 questions correctly.


## Exploring difference

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
print(questions[0])

How does the company recognize the benefit from a tax position?


### Baseline model

In [ ]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[0])

display_response(response)

**`Final Response:`** The company recognizes the benefit from a tax position by including it as a component of income tax expense. They also accrue for the payment of interest and penalties related to unrecognized tax benefits. However, it is important to note that the amounts asserted by tax authorities could be greater or less than the company's accrued position. As a result, the provisions on tax-related matters may change in the future as revised estimates are made or the underlying matters are settled. Additionally, as of January 29, 2023, the company has not identified any positions for which it is reasonably possible that the total amounts of unrecognized tax benefits will significantly increase or decrease within the next twelve months.

### Fine-tuned model

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[0])

display_response(response)

**`Final Response:`** The company acknowledges the advantage of a tax position by incorporating it into their income tax expense. They also categorize an unacknowledged tax benefit as a current liability or a refundable amount if they expect to pay or receive cash for income taxes within a year. If the payment or receipt of cash for income taxes is projected to occur beyond a year, the amount is classified as a long-term liability or a reduction of a long-term refundable amount.